# Environment Setup
This needs to be run the first time you start the notebook

In [1]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer,*-*"

Installed Packages Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.23258.3

Loading extensions from `/Users/Chris.Lumnah/.nuget/packages/microsoft.dotnet.interactive.sqlserver/1.0.0-beta.23258.3/interactive-extensions/dotnet/Microsoft.DotNet.Interactive.SqlServer.dll`

Query Microsoft SQL Server databases. 
 This extension adds support for connecting to Microsoft SQL Server databases using the #!connect mssql magic command. For more information, run a cell using the #!sql magic command.

In [2]:
#!connect mssql --kernel-name TPCCDemo "Persist Security Info=False; TrustServerCertificate=True; Integrated Security=true; Initial Catalog=tpcc_2000gb; Server=rp-sql19sl-01.perf.rubrik.com;"

Kernel added: #!sql-TPCCDemo

In [5]:
# Rubrik Connection Information
$ServiceAccountID = op read op://Rubrik/perfpod-cdm02_ServiceAccount/username
$Secret = op read op://Rubrik/perfpod-cdm02_ServiceAccount/credential
$Server = op read op://Rubrik/perfpod-cdm02_ServiceAccount/hostname

# Source Database Information
$SourceSQLServerInstance = "rp-sql19sl-01.perf.rubrik.com"
$SourceDatabaseName = "TPCC_2000GB"

# Target Database Information
$TargetSQLServerInstance = "rp-sql19sl-01.perf.rubrik.com"
$LiveMountName = "LiveMount"

# Look at the environment before

## List user databases and sizes

- <mark>Pay attention to the database name TPCC\_2000GB</mark>

In [6]:
#!sql-TPCCDemo
with fs
as
(
    select database_id, type, size * 8.0 / 1024 size
    from sys.master_files
)
select
    name,
    (select sum(size) from fs where type = 0 and fs.database_id = db.database_id) DataFileSizeInMB,
    (select sum(size) from fs where type = 1 and fs.database_id = db.database_id) LogFileSizeInMB
from sys.databases db
where db.database_id > 4

(6 rows affected)

name,DataFileSizeInMB,LogFileSizeInMB
TPCC_2000GB,2232762.000000,48577.000000
TPCC_5000GB,5210112.000000,1032.000000
rafa-bulk,16.000000,8.000000
rafa,16.000000,8.000000
rafa-new,16.000000,8.000000
Demo_LM,2232762.000000,48577.000000


## List the tables and their sizes

- <mark>Pay attention to the tables named district and warehouse</mark>
- Look at the row counts for both of these tables

In [7]:
#!sql-TPCCDemo
USE TPCC_2000GB
SELECT 
    t.NAME AS TableName,
    s.Name AS SchemaName,
    p.rows,
    SUM(a.total_pages) * 8 AS TotalSpaceKB, 
    CAST(ROUND(((SUM(a.total_pages) * 8) / 1024.00), 2) AS NUMERIC(36, 2)) AS TotalSpaceMB,
    SUM(a.used_pages) * 8 AS UsedSpaceKB, 
    CAST(ROUND(((SUM(a.used_pages) * 8) / 1024.00), 2) AS NUMERIC(36, 2)) AS UsedSpaceMB, 
    (SUM(a.total_pages) - SUM(a.used_pages)) * 8 AS UnusedSpaceKB,
    CAST(ROUND(((SUM(a.total_pages) - SUM(a.used_pages)) * 8) / 1024.00, 2) AS NUMERIC(36, 2)) AS UnusedSpaceMB
FROM 
    sys.tables t
INNER JOIN      
    sys.indexes i ON t.OBJECT_ID = i.object_id
INNER JOIN 
    sys.partitions p ON i.object_id = p.OBJECT_ID AND i.index_id = p.index_id
INNER JOIN 
    sys.allocation_units a ON p.partition_id = a.container_id
LEFT OUTER JOIN 
    sys.schemas s ON t.schema_id = s.schema_id
WHERE 
    t.NAME NOT LIKE 'dt%' 
    AND t.is_ms_shipped = 0
    AND i.OBJECT_ID > 255 
GROUP BY 
    t.Name, s.Name, p.Rows
ORDER BY 
    TotalSpaceMB DESC, t.Name

(10 rows affected)

TableName,SchemaName,rows,TotalSpaceKB,TotalSpaceMB,UsedSpaceKB,UsedSpaceMB,UnusedSpaceKB,UnusedSpaceMB
stock,dbo,2953200000,949449944,927197.21,949444720,927192.11,5224,5.10
customer,dbo,900000000,654549072,639208.08,654547000,639206.05,2072,2.02
order_line,dbo,9000027799,590169528,576337.43,590167376,576335.33,2152,2.10
history,dbo,900000000,52558336,51326.50,52555200,51323.44,3136,3.06
orders,dbo,900000000,29391024,28702.17,29388448,28699.66,2576,2.52
new_order,dbo,270000000,4816608,4703.72,4810936,4698.18,5672,5.54
district,dbo,300000,2409760,2353.28,2409616,2353.14,144,0.14
warehouse,dbo,30000,241248,235.59,241056,235.41,192,0.19
item,dbo,100000,9616,9.39,9520,9.30,96,0.09
BackupTimeTable,dbo,350,144,0.14,88,0.09,56,0.05


# Do a bad thing

## Delete without a WHERE clause

- A user is doing some work in the database. They need to do some clean up and they want to delete 5 entries from the warehouse table. However instead they made a mistake and left their WHERE clause commented out and mistakenly deleted all records from the table.

In [8]:
#!sql-TPCCDemo
USE TPCC_2000GB
DELETE FROM warehouse
--WHERE W_id IN (1,2,3,4,5)

(30000 rows affected)

## Drop a table

- A Junior DBA is doing a change. They think they are in a non-production environment and mistakenly run this code against production.

In [9]:
#!sql-TPCCDemo
USE TPCC_2000GB
DROP TABLE district

Commands completed successfully.

## List the tables and their sizes

- <mark>Pay attention to the tables named district and warehouse</mark>
- Notice that the warehouse table has 0 records
- Notice the district table no longer exists

In [10]:
#!sql-TPCCDemo
USE TPCC_2000GB
SELECT 
    t.NAME AS TableName,
    s.Name AS SchemaName,
    p.rows,
    SUM(a.total_pages) * 8 AS TotalSpaceKB, 
    CAST(ROUND(((SUM(a.total_pages) * 8) / 1024.00), 2) AS NUMERIC(36, 2)) AS TotalSpaceMB,
    SUM(a.used_pages) * 8 AS UsedSpaceKB, 
    CAST(ROUND(((SUM(a.used_pages) * 8) / 1024.00), 2) AS NUMERIC(36, 2)) AS UsedSpaceMB, 
    (SUM(a.total_pages) - SUM(a.used_pages)) * 8 AS UnusedSpaceKB,
    CAST(ROUND(((SUM(a.total_pages) - SUM(a.used_pages)) * 8) / 1024.00, 2) AS NUMERIC(36, 2)) AS UnusedSpaceMB
FROM 
    sys.tables t
INNER JOIN      
    sys.indexes i ON t.OBJECT_ID = i.object_id
INNER JOIN 
    sys.partitions p ON i.object_id = p.OBJECT_ID AND i.index_id = p.index_id
INNER JOIN 
    sys.allocation_units a ON p.partition_id = a.container_id
LEFT OUTER JOIN 
    sys.schemas s ON t.schema_id = s.schema_id
WHERE 
    t.NAME NOT LIKE 'dt%' 
    AND t.is_ms_shipped = 0
    AND i.OBJECT_ID > 255 
GROUP BY 
    t.Name, s.Name, p.Rows
ORDER BY t.Name

(9 rows affected)

TableName,SchemaName,rows,TotalSpaceKB,TotalSpaceMB,UsedSpaceKB,UsedSpaceMB,UnusedSpaceKB,UnusedSpaceMB
BackupTimeTable,dbo,350,144,0.14,88,0.09,56,0.05
customer,dbo,900000000,654549072,639208.08,654547000,639206.05,2072,2.02
history,dbo,900000000,52558336,51326.50,52555200,51323.44,3136,3.06
item,dbo,100000,9616,9.39,9520,9.30,96,0.09
new_order,dbo,270000000,4816608,4703.72,4810936,4698.18,5672,5.54
order_line,dbo,9000027799,590169528,576337.43,590167376,576335.33,2152,2.10
orders,dbo,900000000,29391024,28702.17,29388448,28699.66,2576,2.52
stock,dbo,2953200000,949449944,927197.21,949444720,927192.11,5224,5.10
warehouse,dbo,0,133472,130.34,133280,130.16,192,0.19


# Recover from the Bad Thing

## Live Mount the last backup

In [11]:
# Connect-Rubrik with a Service Account
Connect-Rubrik -Server $Server -id $ServiceAccountID -Secret $Secret

# Get database information from Rubrik
$RubrikDatabase = Get-RubrikDatabase -Name $SourceDatabaseName -ServerInstance $SourceSQLServerInstance

#Mount a database to a SQL Server
$TargetInstance = Get-RubrikSQLInstance -ServerInstance $TargetSQLServerInstance
$RubrikRequest = New-RubrikDatabaseMount -id $RubrikDatabase.id `
	-TargetInstanceId $TargetInstance.id `
	-MountedDatabaseName $LiveMountName `
	-recoveryDateTime (Get-date (Get-RubrikDatabase -id $RubrikDatabase.id).latestRecoveryPoint) `
    -Confirm:$false
Get-RubrikRequest -id $RubrikRequest.id -Type mssql -WaitForCompletion
Disconnect-Rubrik

VERBOSE: HTTP/1.1 POST with 171-byte payload
VERBOSE: received 580-byte response of content type application/json
VERBOSE: Content encoding: utf-8

Name                           Value
----                           -----
header                         {[User-Agent, RubrikPowerShellSDK-6.0.1--7.3.0--platform--Unix--pla…
authType                       ServiceAccount
userId                         
version                        8.1.1-24187
time                           6/1/2023 9:35:29 AM
server                         10.8.49.101
id                             
api                            1

id        : MSSQL_DB_MOUNT_4b2c0ad1-eff2-4b9b-8b3f-842556aeb0ff_5dc90e68-7fc7-4aaf-bf66-143f038142c
            3:::0
status    : SUCCEEDED
startTime : 6/1/2023 1:35:35 PM
endTime   : 6/1/2023 1:37:10 PM
nodeId    : cluster:::RVMHM204S006647
links     : {@{href=https://10.8.49.101/api/v1/mssql/db/mount/8d5af78d-c35c-4635-b1cb-1885c1b37cc0;
             rel=result}, @{href=https://10.8.49.101/ap

## Look at the environment after

### Look at the files of each database

In [12]:
#!sql-TPCCDemo
USE [TPCC_2000GB]
SELECT
  name 'Logical Name', 
  physical_name 'File Location'
FROM sys.database_files;
USE [livemount]
SELECT
  name 'Logical Name', 
  physical_name 'File Location'
FROM sys.database_files;

(11 rows affected)

(11 rows affected)

Logical Name,File Location
TPCC_01,C:\mnt\sqldata\TPCC_01.mdf
TPCC_log,C:\mnt\sqllogs\TPCC_log.ldf
TPCC_02,C:\mnt\sqldata\TPCC_02.ndf
TPCC_03,C:\mnt\sqldata\TPCC_03.ndf
TPCC_04,C:\mnt\sqldata\TPCC_04.ndf
TPCC_05,C:\mnt\sqldata\TPCC_05.ndf
TPCC_06,C:\mnt\sqldata\TPCC_06.ndf
TPCC_07,C:\mnt\sqldata\TPCC_07.ndf
TPCC_08,C:\mnt\sqldata\TPCC_08.ndf
TPCC_09,C:\mnt\sqldata\TPCC_09.ndf


Logical Name,File Location
TPCC_01,\\10.8.49.102\bzeygjdz2k34o26tvd95067ec\c\mnt\sqldata\TPCC_01.mdf
TPCC_log,\\10.8.49.102\bzeygjdz2k34o26tvd95067ec\c\mnt\sqllogs\TPCC_log.ldf
TPCC_02,\\10.8.49.102\bzeygjdz2k34o26tvd95067ec\c\mnt\sqldata\TPCC_02.ndf
TPCC_03,\\10.8.49.102\bzeygjdz2k34o26tvd95067ec\c\mnt\sqldata\TPCC_03.ndf
TPCC_04,\\10.8.49.102\bzeygjdz2k34o26tvd95067ec\c\mnt\sqldata\TPCC_04.ndf
TPCC_05,\\10.8.49.102\bzeygjdz2k34o26tvd95067ec\c\mnt\sqldata\TPCC_05.ndf
TPCC_06,\\10.8.49.102\bzeygjdz2k34o26tvd95067ec\c\mnt\sqldata\TPCC_06.ndf
TPCC_07,\\10.8.49.102\bzeygjdz2k34o26tvd95067ec\c\mnt\sqldata\TPCC_07.ndf
TPCC_08,\\10.8.49.102\bzeygjdz2k34o26tvd95067ec\c\mnt\sqldata\TPCC_08.ndf
TPCC_09,\\10.8.49.102\bzeygjdz2k34o26tvd95067ec\c\mnt\sqldata\TPCC_09.ndf


### List user databases and sizes

In [13]:
#!sql-TPCCDemo
with fs
as
(
    select database_id, type, size * 8.0 / 1024 size
    from sys.master_files
)
select
    name,
    (select sum(size) from fs where type = 0 and fs.database_id = db.database_id) DataFileSizeInMB,
    (select sum(size) from fs where type = 1 and fs.database_id = db.database_id) LogFileSizeInMB
from sys.databases db
where db.database_id > 4

(7 rows affected)

name,DataFileSizeInMB,LogFileSizeInMB
TPCC_2000GB,2232762.000000,48577.000000
TPCC_5000GB,5210112.000000,1032.000000
rafa-bulk,16.000000,8.000000
rafa,16.000000,8.000000
rafa-new,16.000000,8.000000
Demo_LM,2232762.000000,48577.000000
LiveMount,2232762.000000,48577.000000


## Recover the **warehouse** table

### Put records back

In [14]:
#!sql-TPCCDemo
INSERT INTO TPCC_2000GB.dbo.warehouse
SELECT * FROM livemount.dbo.warehouse

(30000 rows affected)

## Recover the **district** table

### Create the district table

In [15]:
#!sql-TPCCDemo
USE TPCC_2000GB
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [dbo].[district](
	[d_id] [tinyint] NOT NULL,
	[d_w_id] [int] NOT NULL,
	[d_ytd] [money] NOT NULL,
	[d_next_o_id] [int] NULL,
	[d_tax] [smallmoney] NULL,
	[d_name] [char](10) NULL,
	[d_street_1] [char](20) NULL,
	[d_street_2] [char](20) NULL,
	[d_city] [char](20) NULL,
	[d_state] [char](2) NULL,
	[d_zip] [char](9) NULL,
	[padding] [char](6000) NOT NULL
) ON [PRIMARY]
GO
ALTER TABLE [dbo].[district] SET (LOCK_ESCALATION = DISABLE)
GO
ALTER TABLE [dbo].[district] ADD  CONSTRAINT [PK_DISTRICT] PRIMARY KEY CLUSTERED 
(
	[d_w_id] ASC,
	[d_id] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, SORT_IN_TEMPDB = OFF, IGNORE_DUP_KEY = OFF, ONLINE = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [PRIMARY]
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

### Put the records back

In [16]:
#!sql-TPCCDemo
INSERT INTO TPCC_2000GB.dbo.district
SELECT * FROM livemount.dbo.district

(300000 rows affected)

## List the tables and their sizes

In [17]:
#!sql-TPCCDemo
USE TPCC_2000GB
SELECT 
    t.NAME AS TableName,
    s.Name AS SchemaName,
    p.rows,
    SUM(a.total_pages) * 8 AS TotalSpaceKB, 
    CAST(ROUND(((SUM(a.total_pages) * 8) / 1024.00), 2) AS NUMERIC(36, 2)) AS TotalSpaceMB,
    SUM(a.used_pages) * 8 AS UsedSpaceKB, 
    CAST(ROUND(((SUM(a.used_pages) * 8) / 1024.00), 2) AS NUMERIC(36, 2)) AS UsedSpaceMB, 
    (SUM(a.total_pages) - SUM(a.used_pages)) * 8 AS UnusedSpaceKB,
    CAST(ROUND(((SUM(a.total_pages) - SUM(a.used_pages)) * 8) / 1024.00, 2) AS NUMERIC(36, 2)) AS UnusedSpaceMB
FROM 
    sys.tables t
INNER JOIN      
    sys.indexes i ON t.OBJECT_ID = i.object_id
INNER JOIN 
    sys.partitions p ON i.object_id = p.OBJECT_ID AND i.index_id = p.index_id
INNER JOIN 
    sys.allocation_units a ON p.partition_id = a.container_id
LEFT OUTER JOIN 
    sys.schemas s ON t.schema_id = s.schema_id
WHERE 
    t.NAME NOT LIKE 'dt%' 
    AND t.is_ms_shipped = 0
    AND i.OBJECT_ID > 255 
GROUP BY 
    t.Name, s.Name, p.Rows
ORDER BY t.Name

(10 rows affected)

TableName,SchemaName,rows,TotalSpaceKB,TotalSpaceMB,UsedSpaceKB,UsedSpaceMB,UnusedSpaceKB,UnusedSpaceMB
BackupTimeTable,dbo,350,144,0.14,88,0.09,56,0.05
customer,dbo,900000000,654549072,639208.08,654547000,639206.05,2072,2.02
district,dbo,300000,2409760,2353.28,2409616,2353.14,144,0.14
history,dbo,900000000,52558336,51326.50,52555200,51323.44,3136,3.06
item,dbo,100000,9616,9.39,9520,9.30,96,0.09
new_order,dbo,270000000,4816608,4703.72,4810936,4698.18,5672,5.54
order_line,dbo,9000027799,590169528,576337.43,590167376,576335.33,2152,2.10
orders,dbo,900000000,29391024,28702.17,29388448,28699.66,2576,2.52
stock,dbo,2953200000,949449944,927197.21,949444720,927192.11,5224,5.10
warehouse,dbo,30000,241248,235.59,241056,235.41,192,0.19


## Get Rid of the Live Mount

In [18]:
# Connect-Rubrik with a Service Account
Connect-Rubrik -Server $Server -id $ServiceAccountID -Secret $Secret

# Unmount a database from SQL Server
$RubrikDatabaseMount = Get-RubrikDatabaseMount -MountedDatabaseName $LiveMountName -TargetInstanceId $TargetInstance.id
$RubrikRequest = Remove-RubrikDatabaseMount -id $RubrikDatabaseMount.id -Confirm:$false
Disconnect-Rubrik

VERBOSE: HTTP/1.1 POST with 171-byte payload
VERBOSE: received 580-byte response of content type application/json
VERBOSE: Content encoding: utf-8

Name                           Value
----                           -----
header                         {[User-Agent, RubrikPowerShellSDK-6.0.1--7.3.0--platform--Unix--pla…
authType                       ServiceAccount
userId                         
version                        8.1.1-24187
time                           6/1/2023 9:42:18 AM
server                         10.8.49.101
id                             
api                            1
Remove-RubrikDatabaseMount: 
Line |
   6 |  … uest = Remove-RubrikDatabaseMount -id $RubrikDatabaseMount.id -Confir …
     |                                          ~~~~~~~~~~~~~~~~~~~~~~~
     | Cannot process argument transformation on parameter 'id'. Cannot convert value to type System.String.

Status                : Success
HTTPStatusCode        : 204
HTTPStatusDescription : NoContent


